In [1]:
%matplotlib inline
from vnpy.factor.backtesting.backtesting import BacktestEngine
from vnpy.factor.setting import get_backtest_data_cache_path, get_backtest_report_path


backtest_engine = BacktestEngine(
    factor_module_name="vnpy.factor.factors",
    output_data_dir_for_analyser_reports=get_backtest_report_path(),
    output_data_dir_for_calculator_cache=get_backtest_data_cache_path()
)

2025-06-02 04:47:11.374 | INFO     | vnpy.factor.backtesting.backtesting:_write_log:602 - [FactorMakerBacktestOrchestrator] Successfully imported factor module: 'vnpy.factor.factors'
2025-06-02 04:47:11.374 | INFO     | vnpy.factor.backtesting.backtesting:_write_log:602 - [FactorMakerBacktestOrchestrator] FactorMakerBacktestOrchestrator initialized.


[vnpy.trader.setting] Updated SETTINGS from vt_setting.json


In [ ]:
vt_symbols = ['btcusdt.BINANCE', 'ethusdt.BINANCE', 'xrpusdt.BINANCE']

In [2]:
backtest_engine.output_data_dir_for_calculator_cache

PosixPath('/Users/chenzhao/Documents/crypto_vnpy/vnpy/tests/.vnpy/factor/cache/backtest_factor_data_cache')

In [3]:
from vnpy.factor.setting import FACTOR_DEFINITIONS_FILEPATH
from vnpy.factor.utils.factor_utils import load_factor_setting


factor_definations = load_factor_setting(FACTOR_DEFINITIONS_FILEPATH)

In [4]:
macd_factor_defination = factor_definations[2]
macd_factor_defination

{'class_name': 'MACDFactor',
 'factor_name': 'MACDFactor',
 'factor_key': 'factor_1m_macdfactor@fast_period_12-signal_period_9-slow_period_26',
 'freq': '1m',
 'params': {'fast_period': 12, 'slow_period': 26, 'signal_period': 9},
 'dependencies_factor': [{'class_name': 'EMAFactor',
   'factor_name': 'fast_ema',
   'factor_key': 'factor_1m_emafactor@period_12',
   'freq': '1m',
   'params': {'period': 12},
   'dependencies_factor': [],
   'dependencies_freq': [],
   'dependencies_symbol': [],
   'dependencies_exchange': [],
   'last_run_datetime': '2025-05-29 16:26:46',
   'factor_mode': 'LIVE'},
  {'class_name': 'EMAFactor',
   'factor_name': 'slow_ema',
   'factor_key': 'factor_1m_emafactor@period_26',
   'freq': '1m',
   'params': {'period': 26},
   'dependencies_factor': [],
   'dependencies_freq': [],
   'dependencies_symbol': [],
   'dependencies_exchange': [],
   'last_run_datetime': '2025-05-29 16:26:46',
   'factor_mode': 'LIVE'}],
 'dependencies_freq': [],
 'dependencies_symbo

In [5]:
import importlib

from vnpy.factor.utils.factor_utils import init_factors


factor_module = importlib.import_module("vnpy.factor.factors")
macd_factor = init_factors(
    module_for_primary_classes=factor_module,
    settings_data=[macd_factor_defination],
    dependencies_module_lookup_for_instances=factor_module
)[0]

In [6]:
macd_factor.get_nested_params_for_optimizer()

{'fast_period': 12,
 'slow_period': 26,
 'signal_period': 9,
 'fast_ema.period': 12,
 'slow_ema.period': 26}

In [ ]:
backtest_engine._init_and_flatten_factor(macd_factor, vt_symbols)

2025-06-02 04:47:11.455 | INFO     | vnpy.factor.backtesting.backtesting:_write_log:602 - [FactorMakerBacktestOrchestrator] Initializing and flattening factor based on definition. Symbols: ['btcusdt.BINANCE']
2025-06-02 04:47:11.455 | INFO     | vnpy.factor.backtesting.backtesting:_write_log:602 - [FactorMakerBacktestOrchestrator] Target factor instance created: factor_1m_macdfactor@fast_period_12-signal_period_9-slow_period_26
2025-06-02 04:47:11.456 | DEBUG    | vnpy.factor.backtesting.backtesting:_write_log:602 - [FactorMakerBacktestOrchestrator] Factor tree flattened. Total factors in graph: 3


(<vnpy.factor.factors.my_factor_lib.MACDFactor at 0x12604c650>,
 {'factor_1m_emafactor@period_12': <vnpy.factor.factors.my_factor_lib.EMAFactor at 0x105705b20>,
  'factor_1m_emafactor@period_26': <vnpy.factor.factors.my_factor_lib.EMAFactor at 0x105ccf6b0>,
  'factor_1m_macdfactor@fast_period_12-signal_period_9-slow_period_26': <vnpy.factor.factors.my_factor_lib.MACDFactor at 0x12604c650>})

In [8]:
backtest_engine._create_calculator()

2025-06-02 04:50:02.367 | INFO     | vnpy.factor.backtesting.factor_calculator:_write_log:447 - [FactorMakerFactorCalculator] FactorCalculator initialized. Factor cache dir: /Users/chenzhao/Documents/crypto_vnpy/vnpy/tests/.vnpy/factor/cache/backtest_factor_data_cache
2025-06-02 04:50:02.370 | INFO     | vnpy.factor.backtesting.factor_calculator:_write_log:447 - [FactorMakerFactorCalculator] Factor cache directory ensured at: /Users/chenzhao/Documents/crypto_vnpy/vnpy/tests/.vnpy/factor/cache/backtest_factor_data_cache


In [ ]:
from datetime import datetime

import numpy as np
import polars as pl
from typing import List, Dict

# Schema for the intermediate flat DataFrame
_OHLCV_FLAT_SCHEMA = {
    "datetime": pl.Datetime(time_unit="us"),
    "symbol": pl.Utf8,
    "open": pl.Float64,
    "high": pl.Float64,
    "low": pl.Float64,
    "close": pl.Float64,
    "volume": pl.Float64,
}


def generate_fake_ohlcv_wide_dict(
    start_date: datetime,
    end_date: datetime,
    interval: str,
    vt_symbols: List[str]
) -> Dict[str, pl.DataFrame]:
    """
    Generates a dictionary of Polars DataFrames with fake OHLCV data.
    Each key in the dictionary is an OHLCV type ("open", "high", "low", "close", "volume").
    Each DataFrame has a "datetime" column and columns for each symbol in vt_symbols,
    containing the respective OHLCV values.

    Args:
        start_date: The start datetime for the data generation.
        end_date: The end datetime for the data generation.
        interval: Polars interval string for fake data generation (e.g., "1m", "1h").
        vt_symbols: List of symbol strings (e.g., ["BTCUSDT", "ETHUSDT"]).

    Returns:
        Dict[str, pl.DataFrame]: A dictionary where keys are OHLCV types and
                                 values are "wide" DataFrames. Returns an empty
                                 dictionary if no data can be generated.
    """
    if not vt_symbols:
        return {}

    # --- Inlined logic from _generate_fake_ohlcv_flat_df ---
    flat_df: pl.DataFrame
    try:
        fake_dates = pl.datetime_range(
            start=start_date,
            end=end_date,
            interval=interval,
            time_unit="us",
            eager=True,
        )
    except Exception: # Handle potential errors in datetime_range
        return {} # Return empty dict if date range generation fails

    if len(fake_dates) == 0:
        return {} # Return empty dict if no dates are generated

    num_fake_rows_per_symbol = len(fake_dates)
    all_rows_data = []

    for symbol in vt_symbols:
        # Generate somewhat realistic OHLCV data
        close_prices = np.random.rand(num_fake_rows_per_symbol) * 100 + 50
        open_prices = close_prices + (np.random.rand(num_fake_rows_per_symbol) - 0.5) * 10
        high_prices = np.maximum(open_prices, close_prices) + np.random.rand(num_fake_rows_per_symbol) * 5
        low_prices = np.minimum(open_prices, close_prices) - np.random.rand(num_fake_rows_per_symbol) * 5
        low_prices = np.maximum(low_prices, 0.01) # Ensure low is not zero or negative
        volumes = np.random.rand(num_fake_rows_per_symbol) * 1000 + 500

        for i in range(num_fake_rows_per_symbol):
            all_rows_data.append({
                "datetime": fake_dates[i],
                "symbol": symbol,
                "open": open_prices[i],
                "high": high_prices[i],
                "low": low_prices[i],
                "close": close_prices[i],
                "volume": volumes[i],
            })

    if not all_rows_data: # Should only happen if vt_symbols was empty, but that's checked above
        return {}

    flat_df = pl.DataFrame(all_rows_data, schema=_OHLCV_FLAT_SCHEMA)
    # --- End of inlined logic ---

    if flat_df.is_empty(): # Double check, though previous checks should cover this
        return {}

    ohlcv_dict: Dict[str, pl.DataFrame] = {}
    ohlcv_types = ["open", "high", "low", "close", "volume"]

    # Get unique sorted datetimes once for potential error case in pivot
    unique_sorted_datetimes = flat_df.get_column("datetime").unique().sort()

    for ohlcv_type in ohlcv_types:
        try:
            pivoted_df = flat_df.pivot(
                index="datetime",
                on="symbol",  # Changed "columns" to "on"
                values=ohlcv_type
            )
            # Sort by datetime just in case pivoting changes order
            pivoted_df = pivoted_df.sort("datetime")
            ohlcv_dict[ohlcv_type] = pivoted_df
        except Exception as e:
            # Handle potential errors during pivoting
            print(f"Error pivoting data for {ohlcv_type}: {e}. Creating empty structure for this type.")
            # Create an empty DataFrame with the expected structure
            empty_df_for_type = pl.DataFrame({"datetime": unique_sorted_datetimes})
            for sym in vt_symbols:
                 empty_df_for_type = empty_df_for_type.with_columns(pl.lit(None, dtype=pl.Float64).alias(sym))
            ohlcv_dict[ohlcv_type] = empty_df_for_type

    return ohlcv_dict

In [ ]:
from datetime import timedelta


end_dt = datetime.now()
start_dt = end_dt - timedelta(days=5) # Generate for the last 30 minutes
time_interval = "1m" # 5-minute interval

ohlcv_data_dictionary = generate_fake_ohlcv_wide_dict(
    start_date=start_dt,
    end_date=end_dt,
    interval=time_interval,
    vt_symbols=vt_symbols
)
ohlcv_data_dictionary['close']

datetime,BTC/USDT,ETH/USDT,SOL/USDT
datetime[μs],f64,f64,f64
2025-06-02 05:38:51.060889,85.078967,61.039545,77.584026
2025-06-02 05:39:51.060889,95.162016,85.045421,77.733875
2025-06-02 05:40:51.060889,133.736817,135.796074,144.693857
2025-06-02 05:41:51.060889,146.639287,107.764043,54.056126
2025-06-02 05:42:51.060889,85.948108,94.356478,61.02478
…,…,…,…
2025-06-02 06:04:51.060889,126.153961,131.497625,146.398706
2025-06-02 06:05:51.060889,99.113835,100.625613,130.867177
2025-06-02 06:06:51.060889,99.950805,139.428835,112.571748
